In [ ]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
import pickle
import datetime
import matplotlib.ticker as ticker
from scipy.stats import linregress
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
home = os.path.expanduser("~") 
os.chdir(str(project_dir))

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench-runtime"
def get_write_time (algorithm, Ntx, Nty, target, nt, nr):
    command = base_cmd + " " + algorithm + " " + str(Ntx) + " " + str(Nty) + " " + str(target) + " " + str(nt) + " " + str(nr) + " 1 "
    print(command)
    return os.popen(command).read()

In [ ]:
def find_closest_rect(n):
    square = int(n**0.5)
    while n % square != 0 or square % 2 == 1 or (n//square) %2 == 1:
        square -= 1
    return (square, n//square)

def find_closest_solution(n):
    minimum = min(n[0], n[1])
    number_of_desired_traps = int(n[0] * n[1] / 0.6)
    down = number_of_desired_traps 
    up = number_of_desired_traps 
    while down % minimum != 0 or down//minimum % 2 == 1:
        down -= 1
    while up % minimum != 0 or up//minimum % 2 == 1:
        up += 1
    if abs(number_of_desired_traps - down) < abs(number_of_desired_traps - up):
        return (minimum, down // minimum)
    else:
        return (minimum , up // minimum)

def find_closest_two(n):
    rounded = int(n)
    up = rounded 
    down = rounded
    while up % 2 == 1:
        up += 1
    while down % 2 == 1:
        down -= 1
    if abs(n - down) < abs(n - up) and n > 32:
        return down
    else:
        return up

trials = 1000
reps = 10
#atoms_dim = [(2,2), (4,4), (8,8), (12,12), (16,16), (24,24), (32,32)]
atoms_dim = [ (4,4), (8,8), (12,12), (16,16), (24,24), (32,32)]
#traps_dims = [(2,4), (4,8), (8,14), (12, 20), (16, 26), (24, 40), (32, 54)]
traps_dims = [ (4,8), (8,14), (12, 20), (16, 26), (24, 40), (32, 54)]
algorithms = ["REDREC-CPU-V3-2D", "REDREC-GPU-V3-2D", "BIRD-CPU-2D"]

for algorithm in algorithms:
    time_data  = []
    for i in range(len(atoms_dim)):
        print(f"{i}/{len(atoms_dim)}")
        write_time_list = []
        write_time_str = get_write_time(algorithm, traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps)
        time_data += [(float(write_time_str.split(',')[0]), float(write_time_str.split(',')[1]))]
    # Saving the results
    file_name = "timing-batching" + algorithm + "-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
    with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "wb") as file:
        pickle.dump(
            {
                "xvals": [x * y for (x, y) in atoms_dim],
                "algorithm": algorithm,
                "experiment": file_name,
                "data": time_data
            }, file
        )


# Load results

In [ ]:
file_name_cpu = "timing-REDREC-CPU-V3-2D-varying-traps-2024-07-11, 12-25-32.pickle"
file_name_gpu = "timing-REDREC-GPU-V3-2D-varying-traps-2024-07-11, 12-25-51.pickle"
file_name_bird = "timing-BIRD-CPU-2D-varying-traps-2024-07-11, 12-25-53.pickle"
file_name_aro = "timing-ARO-2D-varying-traps-2024-07-11, 18-06-20.pickle"
file_name_cpu_batching = "timing-batchingREDREC-CPU-V3-2D-varying-traps-2024-07-19, 16-44-22.pickle"
file_name_gpu_batching = "timing-batchingREDREC-GPU-V3-2D-varying-traps-2024-07-19, 16-44-50.pickle"
file_name_bird_batching = "timing-batchingBIRD-CPU-2D-varying-traps-2024-07-19, 16-45-07.pickle"

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_bird), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_bird = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_aro), "rb") as file:
    data = pickle.load(file)
    atoms_aro = data["xvals"]
    time_data_aro = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu_batching = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu_batching = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_bird_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_bird_batching = data["data"]

In [ ]:
from scipy.optimize import curve_fit
def xticks(ax, lst):
    ax.set_xticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_xlim(lst[0] - diff, lst[-1] + diff)
def yticks(ax, lst):
    ax.set_yticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_ylim(lst[0] - diff, lst[-1] + diff)

def standard_to_vertex(a, b, c):
    h = -b / (2 * a)
    k = a * (h**2) + b * h + c
    
    return a, h, k

def func(x, a, b, c):
  return  c * x ** 2 

def func_cubic(x, a, b, c, d):
  return d * x ** 3 

def func_quartic(x, a, b, c, d, e):
  return e * x ** 4 


def mark(lst):
  ret = []
  for i in lst:
    if i == "lincpu":
      ret.append("8")
    elif i == "lingpu":
      ret.append("<")
    elif i == "bird":
      ret.append("o")
    elif i == "aro":
      ret.append("s")
    elif i == "rrccpu":
      ret.append("^")
    elif i == "rrcgpu":
      ret.append("v")
    elif i == "rrcbatch":
      ret.append(">")
  return ret

def pal(lst):
  prim = []
  sec = []
  for i in lst:
    if i == "lincpu":
      prim.append("#EDD2EE")
      sec.append("#C46BC7")
    elif i == "lingpu":
      prim.append("#F6B6C7")
      sec.append("#DA1B4E")
    elif i == "bird":
      prim.append("#D9E3FD")
      sec.append("#648EF7")
    elif i == "aro":
      prim.append("#EDD2EE")
      sec.append("#C46BC7")
    elif i == "rrccpu":
      prim.append("#FFF1AD")
      sec.append("#E0BB00")
    elif i == "rrcgpu":
      prim.append("#DAF3CE")
      sec.append("#57B52C")
    elif i == "rrcbatch":
      prim.append("#FFD3AD")
      sec.append("#E06900")
  ret = {'primary': prim, 'secondary': sec}
  return ret


In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]

xvals_batching = [4,8,12,16,24,32] 
yvals_cpu_batching = [x[0]/1e3 for x in time_data_cpu_batching]
yerr_cpu_batching = [x[1]/1e3 for x in time_data_cpu_batching]


fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_cpu, _ = curve_fit(func, xvals, yvals_cpu)
model_cpu_batching, _ = curve_fit(func, xvals_batching, yvals_cpu_batching)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_cpu[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_cpu_batching[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E06900", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals_batching], [yvals_cpu, yvals_cpu_batching], yerr_list=[yerr_cpu, yerr_cpu_batching], markers=mark(["rrccpu", "rrcbatch"]), palette=pal(["rrccpu", "rrcbatch"]),  save_path=str(project_dir) + "/cpu-v-gpu-runtime-2d-batching.pdf")



ax.legend(["Without batching", "With batching"],fontsize=4)
yticks(ax,[0,256,512,768,1024])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-2d-batching.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
yerr_gpu = [x[1]/1e3 for x in time_data_gpu]
yvals_bird = [x[0]/1e3 for x in time_data_bird]
yerr_bird = [x[1]/1e3 for x in time_data_bird]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_bird, _ = curve_fit(func, xvals, yvals_bird)
model_cpu, _ = curve_fit(func, xvals, yvals_cpu)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_cpu[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * -model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_bird[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()
print (model_cpu)
print (model_bird)

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_cpu, yvals_gpu, yvals_bird], yerr_list= [yerr_cpu, yerr_gpu, yerr_bird],markers=mark(["rrccpu", "rrcgpu", "bird"]), palette=pal(["rrccpu", "rrcgpu","bird"]), save_path=str(project_dir) + "/bird-rrccpu-rrcgpu-runtime-2d.pdf")


ax.legend(["Red-rec CPU", "Red-rec GPU", "Bird CPU"],fontsize=4)
yticks(ax, [0, 20,40,60, 80])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("bird-rrccpu-rrcgpu-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
yerr_gpu = [x[1]/1e3 for x in time_data_gpu]


fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_cpu, _ = curve_fit(func, xvals, yvals_cpu)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_cpu[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()



prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals_cpu, yvals_gpu], yerr_list=[yerr_cpu, yerr_gpu],  markers=mark(["rrccpu", "rrcgpu"]), palette=pal(["rrccpu", "rrcgpu"]),save_path=str(project_dir) + "/cpu-v-gpu-runtime-2d.pdf")



ax.legend(["Red-rec CPU", "Red-rec GPU"],fontsize=4)
yticks(ax,[0,15,30,45,60])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
import math
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [math.log(x[0]/1e9, 10) for x in time_data_cpu]
yvals_bird = [math.log(x[0]/1e9, 10) for x in time_data_bird]
yvals_aro = [math.log(x[0]/1e9, 10) for x in time_data_aro]
yvals_gpu = [math.log(x[0]/1e9, 10) for x in time_data_gpu]



fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (s)")

model_cpu, _ = curve_fit(func, xvals, [x[0]/1e3 for x in time_data_cpu])
model_bird, _ = curve_fit(func, xvals, [x[0]/1e3 for x in time_data_bird])
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_aro, _ = curve_fit(func_quartic, xvals, [x[0]/1e3 for x in time_data_aro])
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log(x**2 * model_cpu[2] /1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * -model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log(x**2 * model_bird[2] /1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log(x**4 * model_aro[4] /1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals, xvals], [yvals_cpu, yvals_gpu, yvals_bird, yvals_aro],  markers=mark(["rrccpu", "rrcgpu", "bird", "aro"]), palette=pal(["rrccpu", "rrcgpu", "bird", "aro"]),save_path=str(project_dir) + "/all-runtime-2d.pdf")



ax.legend(["Red-rec CPU","Red-rec GPU", "Bird", "ARO"],fontsize=4)
yticks(ax, [-6, -4,-2,0,2])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))

plt.savefig("all-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = []
yerr_cpu = []
yvals_gpu = []
yerr_gpu = []
yvals_bird = []
yerr_bird = []
yvals_aro = [x[0]/1e9 for x in time_data_aro]
yerr_aro = [x[1]/1e9 for x in time_data_aro]

xvals = [2,4,8,12,16,24,32] 
yvals_cpu = []
yerr_cpu = []
yvals_gpu = []
yerr_gpu = []
yvals_bird = []
yerr_bird = []
yvals_aro = [x[0]/1e9 for x in time_data_aro]
yerr_aro = [x[1]/1e9 for x in time_data_aro]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (s)")

model_aro, _ = curve_fit(func_quartic, xvals, yvals_aro)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**4 * model_aro[4] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals], [yvals_aro], yerr_list=[yerr_aro], markers=mark(["aro"]), palette=pal(["aro"]), save_path=str(project_dir) + "/aro.pdf")

ax.legend(["ARO"],fontsize=4)
ax.set_xlim(-1, 33)
ax.set_ylim(-2, 22)
yticks(ax, [0,5,10,15, 20])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("aro.pdf", format='pdf')
plt.show()
print (model_aro)

In [ ]:
trials = 1000
reps = 100
atoms_dim = [8,16,32,64,128,256,512,768,1024] 
traps_dims = [int(x/0.6) for x in atoms_dim]

algorithms = ["LINEAR-EXACT-GPU-V2-1D", "LINEAR-EXACT-V2-1D"]

for algorithm in algorithms:
    time_data  = []
    for i in range(len(atoms_dim)):
        print(f"{i}/{len(atoms_dim)}")
        write_time_list = []
        write_time_str = get_write_time(algorithm, 1, traps_dims[i], atoms_dim[i], trials, reps)
        time_data += [(float(write_time_str.split(',')[0]), float(write_time_str.split(',')[1]))]
    # Saving the results
    file_name = "timing-" + algorithm + "-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
    with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "wb") as file:
        pickle.dump(
            {
                "xvals": atoms_dim,
                "algorithm": algorithm,
                "experiment": file_name,
                "data": time_data
            }, file
        )

In [ ]:
file_name_cpu = "timing-LINEAR-EXACT-V2-1D-varying-traps-2024-07-10, 15-01-57.pickle"
file_name_gpu = "timing-LINEAR-EXACT-GPU-V2-1D-varying-traps-2024-07-10, 15-00-45.pickle"

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu = data["data"]


In [ ]:
xvals = atoms 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
fig, ax = plt.subplots() 
ax.set_xlabel("Number of traps")
ax.set_ylabel("Time (us)")
result_cpu = linregress(xvals, yvals_cpu)
result_gpu = linregress(xvals, yvals_gpu)
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_cpu.slope + result_cpu.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_gpu.slope + result_gpu.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#DA1B4E", "lw": 1}
).draw()

print("Transfer rate = " + f"{str(round(result_gpu.slope, 6)*10e3)} us/1000atoms")
print("Transfer rate = " + f"{str(round(result_cpu.slope, 6)*10e3)} us/1000atoms")

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals_cpu, yvals_gpu], markers=mark(["lincpu", "lingpu"]),palette=pal(["lincpu", "lingpu"]),  save_path=str(project_dir) + "/cpu-v-gpu-runtime-1d.pdf")
print(pal(["lincpu", "lingpu"]))

ax.legend(["Linear exact CPU", "Linear exact GPU"],fontsize=4)
ax.set_xlim(-50, 1054)
ax.set_ylim(-5, 155)
yticks(ax, [0, 50,100,150])
xticks(ax, [0, 256, 512, 768, 1024])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-1d.pdf", format='pdf')
plt.show()